### Часть 1

In [ ]:
!pip install kaggle
!sudo apt install unzip
!sudo apt-get install p7zip-full

!export KAGGLE_USERNAME='borispanfilov'
!export KAGGLE_KEY='23e1a26705b833184e1835da72c45fe3'

!kaggle competitions download -c avito-context-ad-clicks -f VisitsStream.tsv.7z
!7z x VisitsStream.tsv.7z

In [24]:
! head -n 5 VisitsStream.tsv

UserID	IPID	AdID	ViewDate
59703	1259356	469877	2015-04-25 00:00:00.0
154389	1846749	27252551	2015-04-25 00:00:00.0
218628	2108380	31685325	2015-04-25 00:00:00.0
231535	837110	18827716	2015-04-25 00:00:00.0


In [1]:
# ! yarn application -list 
# ! yarn logs -applicationId application_1740387429795_0005 -log_files stderr | head -n 50

In [47]:
!hdfs dfs -mkdir -p /user/input
!hdfs dfs -mkdir -p /user/output
!hdfs dfs -put VisitsStream.tsv /user/input/VisitsStream.tsv

In [38]:
%%writefile session_mapper.py
import sys
from datetime import datetime

for line in sys.stdin:
    line = line.strip()
    if not line:
        continue
    parts = line.split()
    if len(parts) >= 5:
        user_id = parts[0]
        
        date_str = parts[3] + " " + parts[4]
        dt = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S.%f")

        ts = int(dt.timestamp())
        print(f"{user_id}\t{ts}")

Overwriting session_mapper.py


In [39]:
%%writefile session_reducer.py
import sys

def compute_longest_session(timestamps):
    times = sorted(int(ts) for ts in timestamps)
    
    longest = 0
    session_start = times[0]
    prev = times[0]
    for t in times[1:]:
        if t - prev <= 900:
            prev = t
        else:
            session_duration = prev - session_start
            if session_duration > longest:
                longest = session_duration
            session_start = t
            prev = t
    session_duration = prev - session_start
    if session_duration > longest:
        longest = session_duration
    return longest

current_user = None
timestamps = []

for line in sys.stdin:
    line = line.strip()

    user, ts = line.split('\t', 1)
    if current_user == user:
        timestamps.append(ts)
    else:
        if current_user is not None:
            longest = compute_longest_session(timestamps)
            print(f"{current_user}\t{longest}")
        current_user = user
        timestamps = [ts]

if current_user is not None:
    longest = compute_longest_session(timestamps)
    print(f"{current_user}\t{longest}")


Overwriting session_reducer.py


In [51]:
! hdfs dfs -rm -r /user/output/job1_output || true
! yarn jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Longest Session per User" \
    -D mapreduce.job.reduces=3 \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-k1,1 -n" \
    -D mapreduce.map.output.key.field.separator='\t' \
    -files session_mapper.py,session_reducer.py \
    -mapper "python3 session_mapper.py" \
    -reducer "python3 session_reducer.py" \
    -input /user/input/VisitsStream.tsv \
    -output /user/output/job1_output

Deleted /user/output/job1_output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-3.2.2.jar] /tmp/streamjob6988490524193952341.jar tmpDir=null
2025-02-24 13:15:11,214 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-thq16pdcu3777hh3.mdb.yandexcloud.net/10.128.0.17:8032
2025-02-24 13:15:11,394 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-thq16pdcu3777hh3.mdb.yandexcloud.net/10.128.0.17:10200
2025-02-24 13:15:11,423 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-thq16pdcu3777hh3.mdb.yandexcloud.net/10.128.0.17:8032
2025-02-24 13:15:11,423 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-thq16pdcu3777hh3.mdb.yandexcloud.net/10.128.0.17:10200
2025-02-24 13:15:11,610 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/ubuntu/.staging/job_1740387429795_0009
2025-02-24 13:15:11,873 INFO mapred.FileInputFormat: Total input fi

In [73]:
! hdfs dfs -ls /user/output/job1_output

Found 4 items
-rw-r--r--   1 ubuntu hadoop          0 2025-02-24 13:48 /user/output/job1_output/_SUCCESS
-rw-r--r--   1 ubuntu hadoop   13382398 2025-02-24 13:47 /user/output/job1_output/part-00000
-rw-r--r--   1 ubuntu hadoop   13389507 2025-02-24 13:48 /user/output/job1_output/part-00001
-rw-r--r--   1 ubuntu hadoop   13384421 2025-02-24 13:48 /user/output/job1_output/part-00002


In [84]:
%%writefile top_mapper.py
import sys

for line in sys.stdin:
    user, longest = line.split('\t', 1)
    print(f"{user}+{longest.strip()}\t")

Overwriting top_mapper.py


In [85]:
%%writefile top_reducer.py
import sys

def _rewind_stream(stream):
    for _ in stream:
        pass
    
for _ in range(10):
    key, _ = next(sys.stdin).split('\t')
    user, count = key.split("+")
    print("{}\t{}".format(user, count))
_rewind_stream(sys.stdin)

Overwriting top_reducer.py


In [86]:
! hdfs dfs -rm -r /user/output/job2_output || true
! yarn jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Top 10 Longest Sessions" \
    -D mapreduce.job.reduces=1 \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-k2,2nr k1,1" \
    -D mapreduce.map.output.key.field.separator='+' \
    -files top_mapper.py,top_reducer.py \
    -mapper "python3 top_mapper.py" \
    -reducer "python3 top_reducer.py" \
    -input /user/output/job1_output \
    -output /user/output/job2_output

Deleted /user/output/job2_output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-3.2.2.jar] /tmp/streamjob7768328465098159994.jar tmpDir=null
2025-02-24 14:36:02,025 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-thq16pdcu3777hh3.mdb.yandexcloud.net/10.128.0.17:8032
2025-02-24 14:36:02,205 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-thq16pdcu3777hh3.mdb.yandexcloud.net/10.128.0.17:10200
2025-02-24 14:36:02,233 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-thq16pdcu3777hh3.mdb.yandexcloud.net/10.128.0.17:8032
2025-02-24 14:36:02,234 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-thq16pdcu3777hh3.mdb.yandexcloud.net/10.128.0.17:10200
2025-02-24 14:36:02,431 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/ubuntu/.staging/job_1740387429795_0020
2025-02-24 14:36:02,723 INFO mapred.FileInputFormat: Total input fi

In [87]:
! hdfs dfs -ls /user/output/job2_output

Found 2 items
-rw-r--r--   1 ubuntu hadoop          0 2025-02-24 14:36 /user/output/job2_output/_SUCCESS
-rw-r--r--   1 ubuntu hadoop        142 2025-02-24 14:36 /user/output/job2_output/part-00000


In [88]:
! hdfs dfs -cat /user/output/job2_output/part-00000

4143904	358181
1113291	329463
1203081	170447
4263912	80605
1987990	70544
563902	68359
2853587	53371
2735474	49881
1370797	49791
2271348	48099


In [89]:
!hdfs dfs -get /user/output/job2_output/part-00000 part-00000
!hdfs dfs -put -f part-00000 s3a://bspanfilov-lsml/hw2_part1_top10_users

2025-02-24 14:48:30,244 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2025-02-24 14:48:30,309 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2025-02-24 14:48:30,309 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
2025-02-24 14:48:32,614 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2025-02-24 14:48:32,614 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2025-02-24 14:48:32,614 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


Ссылка: https://storage.yandexcloud.net/bspanfilov-lsml/hw2_part1_top10_users

### Часть 2

In [4]:
# !kaggle competitions download -c avito-context-ad-clicks -f trainSearchStream.tsv.7z
# !kaggle competitions download -c avito-context-ad-clicks -f SearchInfo.tsv.7z
# !kaggle competitions download -c avito-context-ad-clicks -f UserInfo.tsv.7z
# !7z x trainSearchStream.tsv.7z
# !7z x SearchInfo.tsv.7z
# !7z x UserInfo.tsv.7z

In [100]:
! head -n 10 trainSearchStream.tsv

SearchID	AdID	Position	ObjectType	HistCTR	IsClick
2	11441863	1	3	0.001804	0
2	22968355	7	3	0.004723	0
3	212187	7	3	0.029701	0
3	34084553	1	3	0.004300	0
3	36256251	2	2		
4	2073399	6	1		
4	6046052	7	1		
4	17544913	8	1		
4	20653823	1	3	0.003049	0


In [103]:
! head -n 200 SearchInfo.tsv

SearchID	SearchDate	IPID	UserID	IsUserLoggedOn	SearchQuery	LocationID	CategoryID	SearchParams
1	2015-05-18 19:54:32.0	1717090	3640266	0		1729	5	
2	2015-05-12 14:21:28.0	1731568	769304	0		697	50	
3	2015-05-12 07:09:42.0	793143	640089	0		1261	12	
4	2015-05-10 18:11:01.0	898705	3573776	0		3960	22	{83:'Обувь', 175:'Женская одежда', 88:'38'}
5	2015-04-25 13:04:09.0	2009707	320674	0		547	1	
6	2015-05-07 16:49:15.0	1658456	1665156	0		926	27	
7	2015-05-14 23:07:27.0	1849117	3434614	0		44	500001	
8	2015-05-09 09:10:06.0	572585	905821	0		2694	12	
9	2015-05-02 20:14:15.0	77162	1106541	0		576	4	
10	2015-05-15 17:38:46.0	1371072	3310798	0		1110	12	
11	2015-05-17 12:01:01.0	67356	3113486	1		3953	22	{175:'Аксессуары'}
12	2015-05-13 11:54:24.0	1305191	632714	0		1729	4	{156:'Горные'}
13	2015-04-26 21:34:59.0	1473851	638475	0		3960	38	{45:'Кровати, диваны и кресла'}
14	2015-04-28 01:30:16.0	473331	2962661	1		3961	38	{45:'Кровати, диваны и кресла'}
15	2015-05-06 11:23:50.0	1117012	2515153	0		3725	34	
16	

In [5]:
! head -n 5 UserInfo.tsv

UserID	UserAgentID	UserAgentOSID	UserDeviceID	UserAgentFamilyID
1	44073	30	2019	9
2	12505	20	2014	85
3	24256	20	2014	64
4	57133	20	2014	25


In [9]:
# !hdfs dfs -put trainSearchStream.tsv /user/input/trainSearchStream.tsv
# !hdfs dfs -put SearchInfo.tsv /user/input/SearchInfo.tsv
!hdfs dfs -put UserInfo.tsv /user/input/UserInfo.tsv

In [10]:
!hdfs dfs -ls /user/input

Found 6 items
-rw-r--r--   1 ubuntu hadoop         752 2025-02-24 18:19 /user/input/Category.tsv
-rw-r--r--   1 ubuntu hadoop  9469373867 2025-02-24 18:29 /user/input/SearchInfo.tsv
-rw-r--r--   1 ubuntu hadoop   104614699 2025-02-25 08:46 /user/input/UserInfo.tsv
-rw-r--r--   1 ubuntu hadoop 13180996392 2025-02-24 12:01 /user/input/VisitsStream.tsv
-rw-r--r--   1 ubuntu hadoop      459755 2025-02-24 13:06 /user/input/VisitsStream_small.tsv
-rw-r--r--   1 ubuntu hadoop 11023566785 2025-02-24 18:12 /user/input/trainSearchStream.tsv


In [7]:
import findspark
findspark.init()

In [24]:
spark.stop()

### 2.1

In [25]:
from pyspark.sql import SparkSession
import pyspark
from pyspark.sql.functions import col, lower, explode, split, count as spark_count

sc = pyspark.SparkContext(appName="2.1")
spark = SparkSession(sc)

train_search = spark.read.csv("/user/input/trainSearchStream.tsv", sep="\t", header=True)
clicked_search_ids = train_search.filter(col("IsClick") == "1").select("SearchID").distinct()

search_info = spark.read.csv("/user/input/SearchInfo.tsv", sep="\t", header=True)
clicked_search_info = clicked_search_ids.join(search_info, on="SearchID", how="inner")

words = clicked_search_info.select(explode(split(lower(col("SearchQuery")), "\\s+")).alias("word"))

top_words = words.groupBy("word").agg(spark_count("*").alias("count")) \
                 .orderBy(col("count").desc()).limit(10)

top_words.write.csv("/user/output/top10_clicked_words", header=True)
spark.stop()


In [27]:
! hdfs dfs -cat /user/output/top10_clicked_words/part-00000-4f2535af-d192-483f-a69e-3d8ad03ad461-c000.csv

word,count
велосипед,12267
бу,11059
iphone,7314
для,5940
диван,5566
на,4857
платье,4379
коляска,3759
велосипеды,3720
sony,3366


In [28]:
!rm part-00000
!hdfs dfs -get /user/output/top10_clicked_words/part-00000-4f2535af-d192-483f-a69e-3d8ad03ad461-c000.csv part-00000
!hdfs dfs -put -f part-00000 s3a://bspanfilov-lsml/hw2_part2_1_top10_words.csv

2025-02-25 09:13:53,601 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2025-02-25 09:13:53,665 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2025-02-25 09:13:53,665 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
2025-02-25 09:13:56,064 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2025-02-25 09:13:56,064 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2025-02-25 09:13:56,064 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


https://storage.yandexcloud.net/bspanfilov-lsml/hw2_part2_1_top10_words.csv

### 2.2

In [144]:
from pyspark.sql.functions import to_date, countDistinct, lit
from pyspark.sql import Row

sc = pyspark.SparkContext(appName="2.2")
spark = SparkSession(sc)

visits = spark.read.csv("/user/input/VisitsStream.tsv", sep="\t", header=True)

visits = visits.withColumn("date", to_date(col("ViewDate")))

total_days = visits.select("date").distinct().count()

user_days = visits.groupBy("UserID").agg(countDistinct("date").alias("visit_days"))

daily_users = user_days.filter(col("visit_days") == total_days)
num_daily_users = daily_users.count()

result2 = spark.createDataFrame([Row(num_daily_users=num_daily_users)])
result2.write.csv("/user/output/daily_users_count", header=True)

spark.stop()


In [149]:
! hdfs dfs -cat /user/output/daily_users_count/part-00005-84070161-c580-4046-a10f-44e1ec8850ce-c000.csv

num_daily_users
807


In [150]:
!rm part-00000
!hdfs dfs -get /user/output/daily_users_count/part-00005-84070161-c580-4046-a10f-44e1ec8850ce-c000.csv part-00000
!hdfs dfs -put -f part-00000 s3a://bspanfilov-lsml/hw2_part2_2_daily_users.csv

2025-02-24 18:59:49,134 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2025-02-24 18:59:49,203 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2025-02-24 18:59:49,203 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
2025-02-24 18:59:50,748 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2025-02-24 18:59:50,748 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2025-02-24 18:59:50,748 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


https://storage.yandexcloud.net/bspanfilov-lsml/hw2_part2_2_daily_users.csv

### 2.3

In [151]:
sc = pyspark.SparkContext(appName="2.3")
spark = SparkSession(sc)

visits = spark.read.csv("/user/input/VisitsStream.tsv", sep="\t", header=True)
visits = visits.withColumn("date", to_date(col("ViewDate")))
total_days = visits.select("date").distinct().count()
user_days = visits.groupBy("UserID").agg(countDistinct("date").alias("visit_days"))
daily_users = user_days.filter(col("visit_days") == total_days)

daily_user_ids = daily_users.select("UserID").distinct()
visits_filtered = visits.join(daily_user_ids, on="UserID", how="left_anti")
unique_users_per_day = visits_filtered.groupBy("date").agg(countDistinct("UserID").alias("unique_users"))
result3 = unique_users_per_day.orderBy(col("unique_users").desc())

result3.write.csv("/user/output/unique_users_per_day_excluding_daily.csv", header=True)
spark.stop()


In [164]:
! hdfs dfs -ls /user/output/unique_users_per_day_excluding_daily.csv

Found 27 items
-rw-r--r--   1 ubuntu hadoop          0 2025-02-24 21:16 /user/output/unique_users_per_day_excluding_daily.csv/_SUCCESS
-rw-r--r--   1 ubuntu hadoop         36 2025-02-24 21:16 /user/output/unique_users_per_day_excluding_daily.csv/part-00000-57da9514-b67b-41b7-a5c1-d056e7947b81-c000.csv
-rw-r--r--   1 ubuntu hadoop         36 2025-02-24 21:16 /user/output/unique_users_per_day_excluding_daily.csv/part-00001-57da9514-b67b-41b7-a5c1-d056e7947b81-c000.csv
-rw-r--r--   1 ubuntu hadoop         36 2025-02-24 21:16 /user/output/unique_users_per_day_excluding_daily.csv/part-00002-57da9514-b67b-41b7-a5c1-d056e7947b81-c000.csv
-rw-r--r--   1 ubuntu hadoop         36 2025-02-24 21:16 /user/output/unique_users_per_day_excluding_daily.csv/part-00003-57da9514-b67b-41b7-a5c1-d056e7947b81-c000.csv
-rw-r--r--   1 ubuntu hadoop         36 2025-02-24 21:16 /user/output/unique_users_per_day_excluding_daily.csv/part-00004-57da9514-b67b-41b7-a5c1-d056e7947b81-c000.csv
-rw-r--r--   1 ubuntu had

In [160]:
%%writefile put_results_in_one_file_mapper.py
import sys

for line in sys.stdin:
    date, tmp = line.split(',', 1)
    if date != 'date':
        print(f"{date},{tmp}")

Writing put_results_in_one_file_mapper.py


In [161]:
! yarn jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="put results in one file" \
    -D mapreduce.job.reduces=1 \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-k2,2nr" \
    -D mapreduce.map.output.key.field.separator=',' \
    -files put_results_in_one_file_mapper.py \
    -mapper "python3 put_results_in_one_file_mapper.py" \
    -input /user/output/unique_users_per_day_excluding_daily.csv \
    -output /user/output/final_result_2_3

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-3.2.2.jar] /tmp/streamjob80738874967180848.jar tmpDir=null
2025-02-24 21:31:24,265 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-thq16pdcu3777hh3.mdb.yandexcloud.net/10.128.0.17:8032
2025-02-24 21:31:24,470 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-thq16pdcu3777hh3.mdb.yandexcloud.net/10.128.0.17:10200
2025-02-24 21:31:24,507 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-thq16pdcu3777hh3.mdb.yandexcloud.net/10.128.0.17:8032
2025-02-24 21:31:24,508 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-thq16pdcu3777hh3.mdb.yandexcloud.net/10.128.0.17:10200
2025-02-24 21:31:24,705 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/ubuntu/.staging/job_1740387429795_0027
2025-02-24 21:31:24,970 INFO mapred.FileInputFormat: Total input files to process : 26
2025-02-24 21:3

In [166]:
! hdfs dfs -cat /user/output/final_result_2_3/part-00000

2015-05-12,973248	
2015-05-13,847446	
2015-05-14,745904	
2015-05-06,733027	
2015-05-05,731624	
2015-05-07,729708	
2015-05-11,706240	
2015-04-27,676433	
2015-05-08,664982	
2015-04-28,655791	
2015-04-29,641148	
2015-05-15,633132	
2015-05-04,607205	
2015-04-30,596224	
2015-05-10,562151	
2015-04-26,540051	
2015-05-03,511687	
2015-04-25,498069	
2015-05-16,493577	
2015-05-18,488250	
2015-05-17,480968	
2015-05-02,468010	
2015-05-01,452800	
2015-05-09,450135	
2015-05-19,367777	
2015-05-20,184972	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	


In [167]:
!rm part-00000
!hdfs dfs -get /user/output/final_result_2_3/part-00000 part-00000
!hdfs dfs -put -f part-00000 s3a://bspanfilov-lsml/hw2_part2_3_unique_users_per_day_excluding_daily.csv

2025-02-24 21:35:02,004 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2025-02-24 21:35:02,070 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2025-02-24 21:35:02,070 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
2025-02-24 21:35:03,692 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2025-02-24 21:35:03,693 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2025-02-24 21:35:03,693 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


https://storage.yandexcloud.net/bspanfilov-lsml/hw2_part2_3_unique_users_per_day_excluding_daily.csv

### 2.4

In [2]:
!ls

avito-context-ad-clicks.zip	   SearchInfo.tsv      trainSearchStream.tsv
Category.tsv			   SearchInfo.tsv.7z   trainSearchStream.tsv.7z
Category.tsv.7z			   session_mapper.py   VisitsStream_small.tsv
get_results_in_one_file_mapper.py  session_reducer.py  VisitsStream.tsv
HW2_Панфилов_Борис.ipynb	   tmp.py	       VisitsStream.tsv.7z
part-00000			   top_mapper.py
put_results_in_one_file_mapper.py  top_reducer.py


In [17]:
from pyspark.sql import SparkSession
import pyspark
from pyspark.sql.functions import col, lower, explode, split, count, when, sqrt, lit

sc = pyspark.SparkContext(appName="2.4")
spark = SparkSession(sc)

search = spark.read.csv('/user/input/SearchInfo.tsv', sep="\t", header=True)
user = spark.read.csv('/user/input/UserInfo.tsv', sep="\t", header=True)
df = search.join(user, on="UserID", how="left")

N = df.count()

ua_counts = df.groupBy("UserAgentOSID").agg(count("*").alias("ua_count"))
cat_counts = df.groupBy("CategoryID").agg(count("*").alias("cat_count"))
ua_cat_counts = df.groupBy("UserAgentOSID", "CategoryID") \
                              .agg(count("*").alias("ua_cat_count"))

ua_cat = ua_cat_counts.join(ua_counts, on="UserAgentOSID") \
                      .join(cat_counts, on="CategoryID")

ua_cat = ua_cat.withColumn("p_xy", col("ua_cat_count")/lit(N)) \
               .withColumn("p_x", col("ua_count")/lit(N)) \
               .withColumn("p_y", col("cat_count")/lit(N))

# Вычисляем числитель и знаменатель для корреляции
ua_cat = ua_cat.withColumn("numerator", col("p_xy") - col("p_x") * col("p_y")) \
               .withColumn("denom", sqrt(col("p_x")*(1-col("p_x"))*col("p_y")*(1-col("p_y"))))

# Если знаменатель равен 0, то корреляция принимается равной 0
ua_cat = ua_cat.withColumn("correlation", 
                           when(col("denom") != 0, col("numerator")/col("denom")).otherwise(lit(0)))

# Выбираем топ-10 пар по убыванию корреляции
top10_corr = ua_cat.select("UserAgentOSID", "CategoryID", "correlation") \
                   .orderBy(col("correlation").desc()).limit(10)

top10_corr.write.csv("/user/output/top10_correlations", header=True)
spark.stop()


In [19]:
! hdfs dfs -cat /user/output/top10_correlations/part-00000-9731647c-6213-4409-aa2d-ab2c639d43af-c000.csv

UserAgentOSID,CategoryID,correlation
15,0,0.03873487623583474
35,47,0.03524988243205142
30,0,0.02648939747608056
15,16,0.02523947935050731
35,60,0.020480199989959227
28,13,0.01790624393091532
43,50,0.017865609999121113
7,41,0.015962287235910307
15,44,0.015652244860012453
30,41,0.013773293462140119


In [20]:
!rm part-00000
!hdfs dfs -get /user/output/top10_correlations/part-00000-9731647c-6213-4409-aa2d-ab2c639d43af-c000.csv part-00000
!hdfs dfs -put -f part-00000 s3a://bspanfilov-lsml/hw2_part2_4_top10_correlations.csv

2025-02-25 09:03:57,976 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2025-02-25 09:03:58,046 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2025-02-25 09:03:58,046 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
2025-02-25 09:04:01,920 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2025-02-25 09:04:01,920 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2025-02-25 09:04:01,920 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


https://storage.yandexcloud.net/bspanfilov-lsml/hw2_part2_4_top10_correlations.csv